# HW 1

**Fall 2025 Quantiative Methods in Finance**

**The Erdös Institute**

1) Consider the process $X$ of randomly selecting numbers from the unit interval $[0,1]$ until the sum of values exceeds $1$. Prove that $E[X]=e$.

In [27]:
import numpy as np

# Monte Carlo simulation to estimate E[X]
n_trials = 100000
counts = []

for _ in range(n_trials):
    total = 0
    count = 0
    while total <= 1:
        total += np.random.uniform(0, 1)
        count += 1
    counts.append(count)

estimated_e = np.mean(counts)
print(f"Estimated E[X]: {estimated_e:.4f} (True value: {np.e:.4f})")

Estimated E[X]: 2.7253 (True value: 2.7183)


2) Consider the random variable $X$ that selects a number from the two element $\{1,-1\}$ with the uniform distribution until the sum of the drawn values is $1$. Prove that $E[X] = \infty$.

3) Show the probability of winning a pass-line bet in Craps (Vegas rules) is $\frac{244}{495}$.

3) Solution:

**Come-out roll.**  
- Win immediately on 7 or 11: $\frac{6+2}{36}=\frac{8}{36}=\frac{2}{9}$.  
- Lose immediately on 2, 3, or 12: $\frac{1+2+1}{36}=\frac{4}{36}=\frac{1}{9}$.  
- Otherwise a **point** $p\in\{4,5,6,8,9,10\}$ is established.

Let $a(p)$ be the number of dice outcomes that sum to $p$:
$$
a(4)=3,\; a(5)=4,\; a(6)=5,\; a(8)=5,\; a(9)=4,\; a(10)=3,\qquad a(7)=6.
$$

**Given a point $p$,** rolls continue until either $p$ (win) or 7 (loss) appears. The probability of eventually hitting $p$ before 7 is
$$
\mathbb P(\text{make }p\text{ before }7\mid p)=\frac{a(p)}{a(p)+a(7)}=\frac{a(p)}{a(p)+6}.
$$

The probability of establishing $p$ on the come-out and then winning is
$$
\mathbb P(\text{establish }p)\cdot \mathbb P(\text{win}\mid p)=\frac{a(p)}{36}\cdot \frac{a(p)}{a(p)+6}.
$$

**Total win probability**
$$
\begin{aligned}
\mathbb P(\text{win})
&= \underbrace{\frac{8}{36}}_{\text{immediate}} \;+\;
\sum_{p\in\{4,5,6,8,9,10\}}
\frac{a(p)}{36}\cdot \frac{a(p)}{a(p)+6} \\
&= \frac{8}{36}
+ \left[\frac{3}{36}\cdot\frac{3}{9}
+ \frac{4}{36}\cdot\frac{4}{10}
+ \frac{5}{36}\cdot\frac{5}{11}
+ \frac{5}{36}\cdot\frac{5}{11}
+ \frac{4}{36}\cdot\frac{4}{10}
+ \frac{3}{36}\cdot\frac{3}{9}\right] \\
&= \frac{244}{495}\;\approx\;0.49293.
\end{aligned}
$$

Thus, $\boxed{\mathbb P(\text{win on pass line})=\tfrac{244}{495}}$.

4) A gambler has \$256 to gamble with and plays a doubling strategy in a game with a 45% chance of winning and a 1:1 payout:

- They bet \$2. If they win, they stop playing.
- If they lose, they double their previous bet and try again.
- This continues until they either win, in which case they take their profits and walk away, or lose the \$128 bet. At that point their bankroll is exhausted.


a) Find the expected profit of the gambler. 

b) Use Monte-Carlo methods to estimate the expected profit of the gambler.

c) What is the probability the strategy is profitable?

d) Use Monte-Carlo methods to estimate the probability the strategy is profitable.

4) Solution:

### Doubling strategy with $p=0.45$, stakes $2,4,8,16,32,64,128$

**Key observation.** With even-money payout and doubling after each loss, if you win before the 7th loss you always net **$+\$2$**. If you lose all 7 bets, you lose **$-\$255$** (since $2+4+\cdots+128=255$).

Let $p=0.45$ and $q=1-p=0.55$.

- Probability of busting (lose all 7): $\;q^7=0.55^7\approx 0.0152243523$.
- Probability the strategy is profitable (win before bust):  
  $$
  \boxed{\;1-q^7 \approx 0.9847756477\;}\quad\text{(part c)}
  $$
- Expected profit:
  $$
  \mathbb E[\Pi] = (+2)\,(1-q^7)+(-255)\,q^7
  = 2 - 257\,q^7
  \approx \boxed{-1.9126585523}\;\text{dollars.} \quad\text{(part a)}
  $$

So the strategy **wins most of the time** (about 98.48%) but has a **negative expectation** (about $-\$1.91$) because the rare bust is large.

---

### Monte-Carlo (parts b & d)

- Simulate independent sequences of up to 7 bets with success prob $p=0.45$.
- Profit per round $=\;+\$2$ if any success occurs, otherwise $-\$255$.

**Results (1,000,000 rounds; fixed seed):**
- Estimated expected profit: $\;\hat{\mathbb E}[\Pi] \approx -1.9308$  
  (SE $\approx 0.0315$; close to $-1.9127$).
- Estimated profitability probability: $\;\widehat{\mathbb P}(\Pi>0)\approx 0.984705$  
  (SE $\approx 1.23\times10^{-4}$; close to $0.984776$).

These align with the exact values above.


In [28]:
import numpy as np

rng = np.random.default_rng(12345)

def simulate_rounds(n_trials: int, p_win: float = 0.45, max_bets: int = 7):
    # Each row = one gambler's sequence of up to max_bets independent trials
    wins = rng.random((n_trials, max_bets)) < p_win
    any_win = wins.any(axis=1)
    profit = np.where(any_win, 2, -255)  # doubling strategy profit per round
    return profit

# (b,d) Monte Carlo
N = 1_000_000
profits = simulate_rounds(N, p_win=0.45, max_bets=7)

mc_expected_profit = profits.mean()
mc_prob_profitable = (profits > 0).mean()

# Quick 95% CI (normal approx)
se_mean = profits.std(ddof=1) / np.sqrt(N)
se_prob = np.sqrt(mc_prob_profitable * (1 - mc_prob_profitable) / N)

(mc_expected_profit, se_mean, mc_prob_profitable, se_prob)


(np.float64(-1.930815),
 np.float64(0.0315399610526855),
 np.float64(0.984705),
 np.float64(0.00012272352250078202))